In [1]:
import pickle

import numpy as np
import pandas as pd
import torch

import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import torch.optim as optim

from datetime import datetime
from sklearn.model_selection import train_test_split
from torch import nn
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

In [2]:
def my_reshape(tensor):
    return torch.reshape(tensor, (tensor.shape[0], 1))

In [3]:
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер
def simple_preprocessing(batch):
    #spherical_radii = torch.Tensor(batch['spherical_domain_radii'])
    #spherical_radii = my_reshape(spherical_radii)
    
    tags = batch['tags'].long()
    tags = F.one_hot(tags, num_classes=3)
    
    atom_numbers = batch['atomic_numbers'].long()
    atom_numbers = F.one_hot(atom_numbers, num_classes=100)
    
    voronoi_volumes = torch.Tensor(batch['voronoi_volumes'])
    voronoi_volumes = my_reshape(voronoi_volumes)
    
    atom_features = (tags, atom_numbers, voronoi_volumes)#, spherical_radii)
    atom_embeds = torch.cat(atom_features, 1)
    
    edge_index = torch.Tensor(batch['edge_index_new']).long()
    
    distances = torch.Tensor(batch['distances_new'])
    distances = my_reshape(distances)
    
    angles = torch.Tensor(batch['contact_solid_angles'])
    angles = my_reshape(angles)
    
    edges_embeds = torch.cat((distances, angles), 1)
    
    
    return Data(x=atom_embeds.to(device), edge_index=edge_index.to(device), edge_attr=edges_embeds.to(device))

In [4]:
#датасет, который умеет возвращать эелемент и собственную длину
class Dataset(Dataset):

    def __init__(self, data, features_fields, target_field, type_='train', preprocessing=simple_preprocessing):
        
        self.data = data[features_fields]
        self.length = len(data)
        self.target = torch.Tensor(data[target_field].values)
        self.type_ = type_
        self.preprocessing = preprocessing

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        
        system = self.preprocessing(self.data.iloc[index])
        
        if self.type_ == 'train':
            y = self.target[index]
            
            return system, y

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)
$$

Гамма лежит в апдейт, квадратик в aggr, а фи в месседж; в этом примере гамма и фи -- умножение на матрицу после конкатенации, а квадратик -- суммирование

In [5]:
class GConv(MessagePassing):
    def __init__(self, dim_atom=103, dim_edge=2, out_channels=2):
        super(GConv, self).__init__(aggr='add')  # "Add" aggregation
        self.phi_output = 3
        self.lin_phi = torch.nn.Linear(dim_atom*2+dim_edge, self.phi_output, bias=False)
        self.lin_gamma = torch.nn.Linear(dim_atom + self.phi_output, out_channels, bias=False)
        self.nonlin = nn.Sigmoid()

    def forward(self, batch):
        x = batch['x']
        edge_index = batch['edge_index']
        edge_attr = batch['edge_attr']
        
        # x has shape [N -- количество атомов в системе(батче), in_channels -- размерность вектора-атома]
        # edge_index has shape [2, E] -- каждое ребро задаётся парой вершин

        # Start propagating messages. 
    
        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)  #не совсем понял что такое сайз

    def message(self, x, x_i, x_j, edge_attr):
        concatenated = torch.cat((x_i, x_j, edge_attr), 1)
        phi = self.lin_phi(concatenated)
        return self.nonlin(phi)
        
    def update(self, aggr_out, x, edge_attr, edge_index):
                
        concatenated = torch.cat((x, aggr_out), 1)

        return Data(x=self.nonlin(self.lin_gamma(concatenated)), edge_attr=edge_attr, edge_index=edge_index)

In [6]:
#собственно нейросеть
class ConvNN(nn.Module):
    
    def __init__(self, dim_atom=103, dim_edge=2):
        
        super().__init__()          
        self.conv_1 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_2 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_3 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_4 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_5 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_last = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=2)
        
        self.lin = torch.nn.Linear(2, 1, bias=True)
        
    def forward(self, batch):
        convoluted_1 = self.conv_1(batch)
        convoluted_2 = self.conv_2(convoluted_1)
        convoluted_3 = self.conv_3(convoluted_2)
        convoluted_4 = self.conv_4(convoluted_3)
        convoluted_5 = self.conv_5(convoluted_4)
        convoluted_last = self.conv_last(convoluted_5)['x']
        scattered = scatter(convoluted_last, batch['batch'], dim=0, reduce='sum')
        summed = scattered
        energy = self.lin(summed)
        
        return energy

In [7]:
def send_scalars(model, writer):
    pass

In [8]:
def send_hist(model, writer, step):
    for name, weight in model.named_parameters():
        writer.add_histogram(name, weight, step)

In [9]:
#train -- ходим по батчам из итератора, обнуляем градиенты, предсказываем у, считаем лосс, считаем градиенты, делаем шаг оптимайзера, записываем лосс
def train(model, iterator, optimizer, criterion, print_every=10, writer=None):
    
    epoch_loss = 0
    
    model.train()

    for i, (systems, ys) in enumerate(iterator):
        
        optimizer.zero_grad()
        predictions = model(systems).squeeze()
        loss = criterion(predictions.float(), ys.to(device).float())
        loss.backward()     
        
        optimizer.step()      
        
        epoch_loss += loss.item()  
        
        if writer != None:
            send_hist(model, writer, i)
            send_scalars(model, writer)
        
        if not (i+1) % print_every:
            print(f'step {i} from {len(iterator)}')
            print(f'Loss: {epoch_loss/i}')
        
    return epoch_loss / len(iterator)

In [10]:
def evaluate(model, iterator, criterion, writer=False):
    
    epoch_loss = 0
    
#    model.train(False)
    model.eval()  
    
    with torch.no_grad():
        for systems, ys in iterator:   

            predictions = model(systems).squeeze()
            loss = criterion(predictions.float(), ys.to(device).float())        

            epoch_loss += loss.item()
            
            if writer != None:
                send_scalars(optimizer, writer)
                
    print(f'epoch loss {epoch_loss / len(iterator)}')
            
    return epoch_loss / len(iterator)

In [11]:
def inferens(model, iterator):
    y = torch.tensor([])

#    model.train(False)
    model.eval()  
    
    with torch.no_grad():
        for systems, ys in iterator:   
          predictions = model(systemhs).squeeze()
          y = torch.cat((y, predictions))
      
    return y

## DATA

In [12]:
%%time
with open('/Users/humonen/Downloads/structures_train.pkl','rb') as f:
    data_ori = pickle.load(f)

CPU times: user 5.14 s, sys: 1.44 s, total: 6.58 s
Wall time: 6.73 s


In [13]:
%%time
#сливаем новые фичи и фичи из Data
for system in data_ori:
    for key in system['data']:
        system[key[0]] = key[1]
    del system['data']

CPU times: user 134 ms, sys: 4.46 ms, total: 138 ms
Wall time: 137 ms


In [14]:
%%time
df = pd.DataFrame(data_ori)

CPU times: user 74.7 ms, sys: 3.69 ms, total: 78.4 ms
Wall time: 83.6 ms


In [15]:
data_ori=[]

In [16]:
df

,id,voronoi_volumes,voronoi_surface_areas,spherical_domain_radii,distances_new,contact_solid_angles,direct_neighbor,edge_index_new,atomic_numbers,cell,...,edge_index,fixed,force,natoms,pos,pos_relaxed,sid,tags,y_init,y_relaxed
0,0,"[89.3147203199472, 29.342061161368797, 15.7675...","[222.09641997716597, 75.76059098993872, 24.516...","[2.77296770148975, 1.913377530475639, 1.555579...","[4.704193115234375, 4.704193115234375, 2.84391...","[0.45406896877695196, 0.45406896877695196, 9.4...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[[0, 17, 0, 39, 0, 81, 0, 45, 0, 74, 0, 56, 0,...","[tensor(13.), tensor(13.), tensor(13.), tensor...","[[[tensor(11.8325), tensor(0.), tensor(-0.0264...",...,"[[tensor(69), tensor(83), tensor(75), tensor(3...","[tensor(1.), tensor(0.), tensor(1.), tensor(0....","[[tensor(0.0767), tensor(0.0324), tensor(0.578...",86,"[[tensor(7.0256), tensor(0.), tensor(12.7346)]...","[[tensor(7.0256), tensor(0.), tensor(12.7346)]...",2472718,"[tensor(0), tensor(1), tensor(0), tensor(1), t...",6.282501,-0.025550
1,1,"[13.858382002979626, 13.858382236327781, 95.59...","[35.868790697767324, 35.88634058092103, 377.46...","[1.4900744671796733, 1.4900744755429902, 2.836...","[3.6807498931884766, 3.6807498931884766, 3.680...","[0.006754217151977511, 0.006754217151977511, 0...","[0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 8, 0, 3, 0, 52, 0, 11, 0, 48, 0, 55, 0, 6...","[tensor(40.), tensor(40.), tensor(40.), tensor...","[[[tensor(12.7505), tensor(0.), tensor(0.)], [...",...,"[[tensor(55), tensor(63), tensor(52), tensor(4...","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[[tensor(-0.4060), tensor(-0.1663), tensor(-0....",85,"[[tensor(5.3127), tensor(11.3843), tensor(12.7...","[[tensor(5.3127), tensor(11.3843), tensor(12.7...",1747243,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",5.972082,-1.837069
2,2,"[169.90046242317467, 16.909261919823788, 16.90...","[306.2641627020174, 34.77532226895127, 34.6920...","[3.435858874632096, 1.592252773326388, 1.59225...","[3.995115280151367, 3.995115280151367, 4.51798...","[7.348130132770407, 7.348130132770407, 3.46495...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, ...","[[0, 6, 0, 9, 0, 12, 0, 22, 0, 1, 0, 18, 0, 21...","[tensor(47.), tensor(47.), tensor(47.), tensor...","[[[tensor(11.9853), tensor(0.), tensor(0.)], [...",...,"[[tensor(33), tensor(18), tensor(22), tensor(3...","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[[tensor(0.0028), tensor(0.0790), tensor(0.005...",66,"[[tensor(1.9976), tensor(0.), tensor(12.5561)]...","[[tensor(1.9976), tensor(0.), tensor(12.5561)]...",1824821,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",0.686031,-0.152283
3,3,"[35.108055970210486, 22.470958553752013, 25.92...","[73.35361598401914, 33.916565450356856, 44.848...","[2.0312937125432513, 1.7505626701983967, 1.835...","[3.111886501312256, 3.111886501312256, 5.22778...","[15.71836643737836, 15.71836643737836, 0.27114...","[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 53, 0, 38, 0, 29, 0, 36, 0, 48, 0, 9, 0, ...","[tensor(72.), tensor(72.), tensor(72.), tensor...","[[[tensor(8.0230), tensor(0.), tensor(-3.6559)...",...,"[[tensor(9), tensor(53), tensor(45), tensor(36...","[tensor(1.), tensor(1.), tensor(0.), tensor(1....","[[tensor(-0.2585), tensor(-0.0456), tensor(0.0...",62,"[[tensor(9.3824), tensor(12.5032), tensor(14.5...","[[tensor(9.3824), tensor(12.5032), tensor(14.5...",1324413,"[tensor(0), tensor(0), tensor(1), tensor(0), t...",2.222443,-6.227071
4,4,"[38.83699650086656, 14.355720997013236, 14.355...","[165.27906821064923, 34.537843861740484, 34.53...","[2.1008046766541075, 1.5076902742588287, 1.507...","[2.7617592811584473, 2.7617592811584473, 2.761...","[11.216767936903567, 11.216767936903567, 11.21...","[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, ...","[[0, 22, 0, 50, 0, 24, 0, 21, 0, 23, 0, 48, 0,...","[tensor(43.), tensor(43.), tensor(43.), tensor...","[[[tensor(11.0470), tensor(0.), tensor(0.)], [...",...,"[[tensor(18), tensor(16)

In [17]:
df.columns

Index(['id', 'voronoi_volumes', 'voronoi_surface_areas',
       'spherical_domain_radii', 'distances_new', 'contact_solid_angles',
       'direct_neighbor', 'edge_index_new', 'atomic_numbers', 'cell',
       'cell_offsets', 'distances', 'edge_index', 'fixed', 'force', 'natoms',
       'pos', 'pos_relaxed', 'sid', 'tags', 'y_init', 'y_relaxed'],
      dtype='object')

In [18]:
#делим на обучующую и валидационную выборки
df_train, df_val = train_test_split(df, test_size=0.15)
df = []

In [19]:
#сбрасываем индексы
df_train = df_train.reset_index()
df_val = df_val.reset_index()

In [20]:
batch_size = 64
num_workers = 0

In [21]:
# features_cols = ['voloroi_volumes', 'voronoi_surface_areas', 'electronegativity', 
#                  'dipole_polarizability', 'edge_index_new', 'distances_new', 'contact_solid_angles']

features_cols = ['atomic_numbers', 'edge_index_new', 'distances_new', 
                 'contact_solid_angles', 'tags', 'voronoi_volumes', 'spherical_domain_radii']
target_col = 'y_relaxed'

In [22]:
#инициализируем тренировочный датасети и тренировочный итератор
training_set = Dataset(df_train, features_cols, target_col)
training_generator = DataLoader(training_set, batch_size=batch_size, num_workers=num_workers)

In [23]:
#инициализируем валидационный датасет и валидационный итератор
valid_set = Dataset(df_val, features_cols, target_col)
valid_generator = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [24]:
df_train = []
df_val = []

## MODEL

In [25]:
#чтобы тензор по умолчанию заводился на куде
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('cuda')

In [26]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cpu


In [27]:
#model
model = ConvNN(dim_atom=training_set[0][0].x.shape[1],dim_edge=training_set[0][0].edge_attr.shape[1])

#optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

In [28]:
optimizer.state_dict

<bound method Optimizer.state_dict of Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)>

In [29]:
dir(optimizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_hook_for_profile',
 '_zero_grad_profile_name',
 'add_param_group',
 'defaults',
 'load_state_dict',
 'param_groups',
 'state',
 'state_dict',
 'step',
 'zero_grad']

In [30]:
timestamp = str(datetime.now()).split('.')[0]

In [31]:
#tensorboard writer, при первом запуске надо руками сделать папку для логов
writer = SummaryWriter('tensorboard_logs' + '/' + timestamp)

In [32]:
#граф модели
trace_system = dict(list(next(iter(training_generator))[0]))
writer.add_graph(model, trace_system)

## Training

In [ ]:
%%time
loss = []
loss_eval = []
epochs = 10
print(timestamp)
print(f'Start training model {str(model)}')
for i in range(epochs):
    print(f'epoch {i}')
    loss.append(train(model, training_generator, optimizer, criterion, writer=writer))
    loss_eval.append(evaluate(model, valid_generator, criterion, writer=writer))

2021-09-02 14:20:21
Start training model ConvNN(
  (conv_1): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_2): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_3): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_4): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_5): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_last): GConv(
    

In [ ]:
writer.close()

In [ ]:
loss_eval